# Example Seldon Core Deployments using Ksonnet


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

** If running on GCP then run following to add cluster-admin to your user account **

In [12]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

clusterrolebinding.rbac.authorization.k8s.io/my-cluster-admin-binding created


On most clusters you will probably need to add cluster-admin privledges

In [13]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [14]:
!kubectl create namespace seldon

namespace/seldon created


In [15]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [16]:
!rm -rf my-ml-deployment && ks init my-ml-deployment 

INFO Using context "minikube" from kubeconfig file "/home/clive/.kube/config" 
INFO Creating environment "default" with namespace "seldon", pointing to "version:v1.10.0" cluster at address "https://192.168.39.132:8443" 
INFO Generating ksonnet-lib data at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/lib/ksonnet-lib/v1.10.0' 


In [17]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --namespace=seldon --withRbac=true

INFO Retrieved 16 files                           
INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/seldon-core.jsonnet' 


In [18]:
!cd my-ml-deployment && \
      ks apply default

INFO Applying serviceaccounts seldon.seldon       
INFO Creating non-existent serviceaccounts seldon.seldon 
INFO Applying clusterroles seldon-crd             
INFO Creating non-existent clusterroles seldon-crd 
INFO Applying roles seldon.seldon-local           
INFO Creating non-existent roles seldon.seldon-local 
INFO Applying rolebindings seldon.seldon          
INFO Creating non-existent rolebindings seldon.seldon 
INFO Applying clusterrolebindings seldon.seldon   
INFO Creating non-existent clusterrolebindings seldon.seldon 
INFO Applying deployments seldon.seldon-core-seldon-cluster-manager 
INFO Creating non-existent deployments seldon.seldon-core-seldon-cluster-manager 
INFO Applying deployments seldon.seldon-core-redis 
INFO Creating non-existent deployments seldon.seldon-core-redis 
INFO Applying services seldon.seldon-core-redis   
INFO Creating non-existent services seldon.seldon-core-redis 
INFO Applying customresourcedefinitions seldondeployments.machinelearning.seldon.io

In [19]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [1]:
!cp ../proto/prediction.proto ./proto
!cd ../proto/tensorflow && make create_protos
!cp -vr ../proto/tensorflow/tensorflow .
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

make: Nothing to be done for 'create_protos'.
'../proto/tensorflow/tensorflow/core/framework/types.proto' -> './tensorflow/core/framework/types.proto'
'../proto/tensorflow/tensorflow/core/framework/resource_handle.proto' -> './tensorflow/core/framework/resource_handle.proto'
'../proto/tensorflow/tensorflow/core/framework/tensor_shape.proto' -> './tensorflow/core/framework/tensor_shape.proto'
'../proto/tensorflow/tensorflow/core/framework/tensor.proto' -> './tensorflow/core/framework/tensor.proto'


## Serve Single Model

In [21]:
!cd my-ml-deployment && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 && \
    ks apply default -c mymodel

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymodel.jsonnet' 
INFO Applying seldondeployments seldon.mymodel    
INFO Creating non-existent seldondeployments seldon.mymodel 


In [11]:
!cd my-ml-deployment && \
    ks show default -c mymodel

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymodel
  name: mymodel
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymodel
  name: mymodel
  predictors:
  - annotations:
      predictor_version: v1
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: mymodel
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children: []
      endpoint:
        type: REST
      name: mymodel
      type: MODEL
    name: mymodel
    replicas: 1


In [22]:
!kubectl rollout status deploy/mymodel-mymodel-025d03d

Waiting for deployment "mymodel-mymodel-025d03d" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-025d03d" successfully rolled out


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [23]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:mymodel-mymodel-025d03d replicas:1 replicasAvailable:1]] state:Available]

### Get predictions

In [24]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [25]:
r = rest_request_ambassador("mymodel",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "6fu055gjg4guo0mis6833k18lj",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "mymodel": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.0981152186706275]
    }
  }
}


#### gRPC Request

In [26]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "gphkckj2ssr8rtqpvi9q6rukej"
  requestPath {
    key: "mymodel"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09775303765952643
  }
}

## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [27]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

service/example-auth created
deployment.extensions/example-auth created


** Need to wait until running before adding Ambassador config **

In [29]:
!kubectl get pods -n seldon

NAME                                                  READY   STATUS      RESTARTS   AGE
alertmanager-deployment-75f466c76f-mcmn5              1/1     Running     0          4m
example-auth-776ddb4c48-tppsz                         1/1     Running     0          11s
grafana-prom-deployment-875855d77-4d2sp               1/1     Running     0          4m
grafana-prom-import-dashboards-nwtb5                  0/1     Completed   0          4m
mymodel-mymodel-025d03d-7fcdc4454f-qcztz              2/2     Running     0          4m
prometheus-deployment-55657bcc7c-r95g4                1/1     Running     0          4m
prometheus-node-exporter-r2w49                        1/1     Running     0          4m
seldon-core-ambassador-778c58bf5d-ncc5m               2/2     Running     0          5m
seldon-core-redis-5c498fc545-qjkw9                    1/1     Running     0          5m
seldon-core-seldon-cluster-manager-66b6b546d4-2h4lc   1/1     Running     0          5m


In [30]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

service/example-auth configured


Show failed request when auth is running

In [32]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

<Response [401]>

Show successful request with auth

In [33]:
r = rest_request_ambassador_auth("mymodel","username","password",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "njghshgs42oqen1plobd0tqfv2",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "mymodel": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07268601161215003]
    }
  }
}


In [34]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" deleted
deployment.extensions "example-auth" deleted


In [35]:
!cd my-ml-deployment && \
    ks delete default -c mymodel && \
    ks component rm mymodel

INFO Deleting seldondeployments seldon.mymodel    
INFO removing environment component                component-name=mymodel
INFO Removing component parameter references ...  
INFO Deleting component 'mymodel' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymodel.jsonnet' 
INFO Successfully deleted component 'mymodel'     


## Serve AB Test

In [36]:
!cd my-ml-deployment && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c myabtest

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myabtest.jsonnet' 
INFO Applying seldondeployments seldon.myabtest   
INFO Creating non-existent seldondeployments seldon.myabtest 


In [37]:
!cd my-ml-deployment && \
    ks show default -c myabtest

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: myabtest
  name: myabtest
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: myabtest
  name: myabtest
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children:
      - children: []
        endpoint:
          type: RES

In [38]:
!kubectl rollout status deploy/myabtest-myabtest-41de5b8
!kubectl rollout status deploy/myabtest-myabtest-df66c5c

deployment "myabtest-myabtest-41de5b8" successfully rolled out
deployment "myabtest-myabtest-df66c5c" successfully rolled out


In [39]:
!kubectl get seldondeployments myabtest -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:myabtest-myabtest-41de5b8 replicas:1 replicasAvailable:1] map[name:myabtest-myabtest-df66c5c replicas:1 replicasAvailable:1]] state:Available]

### Get predictions

In [40]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [41]:
r = rest_request_ambassador("myabtest",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "stst7hvj3943lv5joj1j57svjn",
    "tags": {
    },
    "routing": {
      "random-ab-test": 1
    },
    "requestPath": {
      "random-ab-test": "",
      "classifier-2": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08565122833269298]
    }
  }
}


#### gRPC Request

In [43]:
grpc_request_ambassador("myabtest",API_AMBASSADOR)

meta {
  puid: "52m5i1ubo7j94r0b2vkjejcp23"
  routing {
    key: "random-ab-test"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "random-ab-test"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07280108134760398
  }
}

In [44]:
!cd my-ml-deployment && \
    ks delete default -c myabtest && \
    ks component rm myabtest

INFO Deleting seldondeployments seldon.myabtest   
INFO removing environment component                component-name=myabtest
INFO Removing component parameter references ...  
INFO Deleting component 'myabtest' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myabtest.jsonnet' 
INFO Successfully deleted component 'myabtest'    


## Serve Multi-Armed Bandit

In [45]:
!cd my-ml-deployment && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c mymab

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymab.jsonnet' 
INFO Applying seldondeployments seldon.mymab      
INFO Creating non-existent seldondeployments seldon.mymab 


In [46]:
!cd my-ml-deployment && \
    ks show default -c mymab

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymab
  name: mymab
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymab
  name: mymab
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v1
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - spec:
        containers:
        - image: seldonio/mab_epsilo

In [47]:
!kubectl rollout status deploy/mymab-mymab-41de5b8
!kubectl rollout status deploy/mymab-mymab-b8038b2
!kubectl rollout status deploy/mymab-mymab-df66c5c

Waiting for deployment "mymab-mymab-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-mymab-41de5b8" successfully rolled out
deployment "mymab-mymab-b8038b2" successfully rolled out
deployment "mymab-mymab-df66c5c" successfully rolled out


In [48]:
!kubectl get seldondeployments mymab -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:mymab-mymab-41de5b8 replicas:1 replicasAvailable:1] map[name:mymab-mymab-df66c5c replicas:1 replicasAvailable:1] map[replicasAvailable:1 name:mymab-mymab-b8038b2 replicas:1]] state:Available]

### Get predictions

In [49]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [50]:
r = rest_request_ambassador("mymab",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "52n808kpb2km7qh7da8k65e7ee",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.0906984954025307]
    }
  }
}


#### gRPC Request

In [51]:
grpc_request_ambassador("mymab",API_AMBASSADOR)

meta {
  puid: "e5lvpd5g1pjc53uddkc0eb47k5"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08545981031608306
  }
}

In [52]:
!cd my-ml-deployment && \
    ks delete default -c mymab && \
    ks component rm mymab

INFO Deleting seldondeployments seldon.mymab      
INFO removing environment component                component-name=mymab
INFO Removing component parameter references ...  
INFO Deleting component 'mymab' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymab.jsonnet' 
INFO Successfully deleted component 'mymab'       


## Serve Model with Outlier Detector

In [53]:
!cd my-ml-deployment && \
    ks generate seldon-outlier-detector-v1alpha2 myout --image seldonio/mock_classifier:1.0 && \
    ks apply default -c myout

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myout.jsonnet' 
INFO Applying seldondeployments seldon.myout      
INFO Creating non-existent seldondeployments seldon.myout 


In [54]:
!cd my-ml-deployment && \
    ks show default -c myout

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: myout
  name: myout
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: myout
  name: myout
  predictors:
  - annotations:
      predictor_version: v1
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: myout
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - spec:
        containers:
        - image: seldonio/outlier_mahalanobis:0.3
          imagePullPolicy: IfNotPresent
          name: outlier-detector
        terminationGracePeriodSeconds: 1
    graph:
      children:
      - children: []
        endpoint:
          type: REST
        name: myout
        type: MODEL
      endpoint:
        type: REST
      name: outlier-detector
      type: TRA

In [55]:
!kubectl rollout status deploy/myout-myout-7551cf4
!kubectl rollout status deploy/myout-myout-d03fe8b

Waiting for deployment "myout-myout-7551cf4" rollout to finish: 0 of 1 updated replicas are available...
deployment "myout-myout-7551cf4" successfully rolled out
deployment "myout-myout-d03fe8b" successfully rolled out


In [56]:
!kubectl get seldondeployments myout -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:myout-myout-7551cf4 replicas:1] map[replicas:1 replicasAvailable:1 name:myout-myout-d03fe8b]] state:Available]

### Get predictions

In [57]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [58]:
r = rest_request_ambassador("myout",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "kqsuror782a0dhfh3k0htm6lvu",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    },
    "requestPath": {
      "outlier-detector": "seldonio/outlier_mahalanobis:0.3",
      "myout": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07669968580977957]
    }
  }
}


#### gRPC Request

In [59]:
grpc_request_ambassador("myout",API_AMBASSADOR)

meta {
  puid: "dhbvq4qusm01399lr5pkfqdr5l"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 0.0
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
  requestPath {
    key: "myout"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "outlier-detector"
    value: "seldonio/outlier_mahalanobis:0.3"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07192904687724741
  }
}

In [60]:
!cd my-ml-deployment && \
    ks delete default -c myout && \
    ks component rm myout

INFO Deleting seldondeployments seldon.myout      
INFO removing environment component                component-name=myout
INFO Removing component parameter references ...  
INFO Deleting component 'myout' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myout.jsonnet' 
INFO Successfully deleted component 'myout'       


# Tear down

In [61]:
!cd my-ml-deployment && ks delete default

INFO Deleting serviceaccounts seldon.seldon       
INFO Deleting clusterroles seldon-crd             
INFO Deleting roles seldon.seldon-local           
INFO Deleting rolebindings seldon.seldon          
INFO Deleting clusterrolebindings seldon.seldon   
INFO Deleting deployments seldon.seldon-core-seldon-cluster-manager 
INFO Deleting deployments seldon.seldon-core-redis 
INFO Deleting services seldon.seldon-core-redis   
INFO Deleting customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Deleting roles seldon.ambassador             
INFO Deleting rolebindings seldon.ambassador      
INFO Deleting deployments seldon.seldon-core-ambassador 
INFO Deleting services seldon.seldon-core-ambassador 


In [62]:
!rm -rf my-ml-deployment